# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


df = pd.read_csv('homepage_actions.csv')
#932 clicked in control group; 4264 in control.
#928 clicked in experiemnt group; 39263 in experiment. 
# People that clicked but didn't view = Type I error? False positive? Can control
# this by setting an alpha level of 0.05

df.loc[(df['group'] == 'experiment') & (df['action'] == 'click')]
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
"""
Ho = There is no difference between the response rate (number of clicks) between
the experimental and control groups.

Ha = There is a difference in response rates (number of clicks) between the experimental
and control groups, with experimental group getting more clicks than the control.

Since we are unsure whether the experimental group is more effective,
a one-sided, two sample t-test will be conducted to compare the differences between
the two means of each group.
"""

'\nHo = There is no difference between the response rate (number of clicks) between\nthe experimental and control groups.\n\nHa = There is a difference in response rates (number of clicks) between the experimental\nand control groups, with experimental group getting more clicks than the control.\n\nSince we are unsure whether the experimental group is more effective,\na one-sided, two sample t-test will be conducted to compare the differences between\nthe two means of each group.\n'

In [3]:
df.head()
f1 = 'group'
f2 = 'action'
f3 = 'id'
f1c1 = 'experiment'
f1c2 = 'control'

#To make the values ready for analysis, view and click we be changed to the integers
# 0 and 1 respectively
#df[f2].replace({'view': 0, 'click': 1}, inplace=True)


In [4]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [5]:
#use .pivot to create table with unique ids only along with the accompanying values
# for click and view
cont = df.loc[df[f1] == f1c2].pivot(index=f3, columns=f2, values='count')
#Since some values are null, they can be filled in with 0s using the fillna method
cont = cont.fillna(value=0)

exper = df.loc[df[f1] == f1c1].pivot(index=f3, columns=f2, values= 'count')
exper = exper.fillna(value=0)

In [6]:
exper.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [7]:
print(f"""
Sample Size: 
\tExperiment: {len(exper)} \tControl: {len(cont)} \n
Number of clicks:
\tExperiment: {sum(exper.click)} \tControl: {sum(cont.click)}\n
Percentage that clicked:
\tExperiment: {round(np.mean(exper.click),2)} \tControl: {round(np.mean(cont.click),2)}
""")



Sample Size: 
	Experiment: 2996 	Control: 3332 

Number of clicks:
	Experiment: 928.0 	Control: 932.0

Percentage that clicked:
	Experiment: 0.31 	Control: 0.28



In [8]:
import scipy.stats as stats

results = stats.ttest_ind(exper.click, cont.click, equal_var=False)

t = results[0]
p = results[1]/2
alpha = 0.05

if p <= alpha:
    print(f"The results are significant with a p-value of {round(p,4)}")
else:
    print(f"The results are not significant with a p-value of {p}")

The results are significant with a p-value of 0.0045


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [9]:
#Your code here
cont_mean = (np.mean(cont.click))
Ho_experiment = len(exper) * cont_mean
Ho_experiment

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
n = len(exper)
expected_p = cont_mean
var = n * expected_p * (1-expected_p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
exper_clicks = sum(exper.click)

z = (exper_clicks - Ho_experiment)/std
z

3.6625360854823588

In [12]:
p2 = stats.norm.sf(z)
p2 - p 

-0.004341537534267589

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
"""
The difference between the raw p-value calculated and the p-value derived from the
t-test is 0.004. The difference is marginal and both p-values are significant. 
Since both tests produced significant values, the Ho can be rejected and it can be
concluded that the experimental page did produce more clicks and was therefore more effective. 
"""


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.